# Quantitative Asset Management
## Assignment 1
## Problem D

In [24]:
import numpy as np
import pandas as pd
import datetime

import pandas_datareader as web

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Define BlackLitterman function

In [25]:
def blacklitterman(delta, w_M, Omega, tau, P, Q, U=None):
    """
    This function computes the Black-Litterman posterior risk premia and variance-covariance matrix

    Inputs:
        delta  - Variance aversion of the average investor
        w_M    - Nx1 vector of weights of the assets in the benchmark portfolio
        Omega  - NxN Prior covariance matrix
        tau    - Coefficient of uncertainty in the prior estimate of the mean (mu)
        P      - KxN matrix for the view(s)
        Q      - Kx1 Vector of view returns
        U      - KxK Matrix of variance of the views (diagonal)
                 - If U is left unspecified, then it is computed
                   as a matrix where the element corresponding to the
                   k-th view is u_k = tau*variance of view portfolio k.
                   That is, the matrix U is given by the formula
             (1)      U = P * tau * Omega * P' .* eye(K,K)

    Outputs:
        mubar    - Nx1 vector of posterior risk premia
        Omegabar - NxN posterior covariance matrix
        w_star - Nx1 Unconstrained weights of the portfolio computed
                     given the posterior estimates mubar and Omega

        Lambda - this is the Kx1 vector of weights received by the view portfolios in the w_star portfolio.
                 Note that w_star can be written as   (2) w_star = inv(1+tau)*(w_M+P'*Lambda)
    """

    # Back out the equilibrium returns mueq
    K = Q.shape[0]
    mueq = delta * (Omega @ w_M)
    # Compute tau * Omega once
    tO = tau * Omega

    if U is not None:
        # Compute posterior mean
        inv_term = np.linalg.inv(P @ tO @ P.T + U)
        mubar = mueq + tO @ P.T @ inv_term @ (Q - P @ mueq)

        # Compute posterior uncertainty in the mean
        pA = tO - tO @ P.T @ inv_term @ (P @ tO)

        # Compute posterior covariance
        # Alternate formula: Omegabar = Omega + pA
        Omegabar = Omega

        # Compute posterior weights
        w_star = np.linalg.inv(delta * Omegabar) @ mubar

        # Compute Lambda solving from formula (2) above
        # Alternate formula: Lambda_val = np.linalg.pinv(P.T) @ (w_star * (1 + tau) - w_M)
        Lambda_val = np.linalg.pinv(P.T) @ (w_star - w_M)

    else:
        # This uses the usual formula for U
        U = P @ tO @ P.T * np.eye(K)
        inv_term = np.linalg.inv(P @ tO @ P.T + U)
        mubar = mueq + tO @ P.T @ inv_term @ (Q - P @ mueq)
        # pA = tO - tO @ P.T @ inv_term @ (P @ tO)
        # Omegabar = Omega + pA
        Omegabar = Omega
        w_star = np.linalg.inv(delta * Omegabar) @ mubar
        # Alternative formula: Lambda_val = np.linalg.pinv(P.T) @ (w_star * (1 + tau) - w_M)
        Lambda_val = np.linalg.pinv(P.T) @ (w_star - w_M)

    return mubar, Omegabar, w_star, Lambda_val, U

## Load Data

In [26]:
#Load the data on Industries

data = pd.read_csv('Data_Industries.txt',  delim_whitespace=True) # Read the CSV file, assuming the delimiter is whitespace.
data['Year'] = pd.to_datetime(data['Year'], format='%Y') # Since the 'Year' column is already present, set it as the index.
data.set_index('Year', inplace=True)
data.index = pd.to_datetime(data.index, format='%Y')
data.index = data.index.year
data = data / 100 # Convert the returns from percentages to a decimal format by dividing by 100.

# Display the first few rows of the DataFrame to verify
data

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Year,,,,,,,,,,
1927,0.2674,0.2264,0.3893,0.1108,0.4078,0.2581,0.2854,0.5654,0.4270,0.3734
1928,0.2675,0.5253,0.5235,0.4246,0.6958,0.2154,0.5018,0.3258,0.5822,0.3057
1929,-0.3731,-0.5549,-0.3208,-0.3127,-0.0845,0.0426,-0.4306,-0.1751,0.1823,-0.2202
1930,-0.2903,-0.4267,-0.4016,-0.4731,-0.4423,-0.0738,-0.4080,-0.1787,-0.2214,-0.3968
1931,-0.2840,-0.4269,-0.5024,-0.4679,-0.5087,-0.3671,-0.4095,-0.3721,-0.3902,-0.5303
...,...,...,...,...,...,...,...,...,...,...
2017,0.1256,0.1415,0.1800,-0.1254,0.2739,0.1465,0.0200,0.2549,0.1158,0.1446
2018,-0.0793,-0.2814,-0.2109,-0.3695,-0.0555,-0.1531,-0.1306,-0.2248,0.0301,-0.1364
2019,0.1568,0.2514,0.2181,-0.1800,0.3034,0.0632,0.1598,0.2596,0.2086,0.2637


In [27]:
# Load the data on risk-free rate
rf = pd.read_csv('Data_Riskfree.txt', delim_whitespace=True)
rf.rename(columns={'Risk': 'Risk Free Rate'}, inplace=True)
rf = rf[['Year', 'Risk Free Rate']]
rf.set_index('Year', inplace=True)
rf.index = pd.to_datetime(rf.index, format='%Y')
rf.index = rf.index.year
rf = rf/100

# Part 1)

In [28]:
T = data.shape[0]
N = data.shape[1]
# Compute excess returns
re = data.sub(rf['Risk Free Rate'], axis=0)
Omega = re.cov()
mu = re.mean()

weights = np.array([6.90, 1.30, 11.96, 9.61, 19.14, 5.43, 9.81, 9.32, 4.46, 22.07])
w_M = weights / 100

In [29]:
EM = w_M @ mu
VarM = w_M.T @ Omega @ w_M   
delta = EM / VarM
tau = 1 / T
mueq = delta * (Omega @ w_M)

print(f"Sample mean excess return: {EM:.4f}")
print(f"Sample variance: {VarM:.4f}")
print(f"Variance aversion coefficient (delta): {delta:.4f}")
print(f"Prior uncertainty parameter (tau): {tau:.4f}")
print(f"Equilibrium risk premia (μ_eq): \n{mueq}")

Sample mean excess return: 0.1448
Sample variance: 0.0913
Variance aversion coefficient (delta): 1.5859
Prior uncertainty parameter (tau): 0.0105
Equilibrium risk premia (μ_eq): 
NoDur    0.131504
Durbl    0.175123
Manuf    0.149721
Enrgy    0.132929
HiTec    0.180835
Telcm    0.127861
Shops    0.154195
Hlth     0.119837
Utils    0.069353
Other    0.144029
dtype: float64


# Part 2)

In [30]:
## Black-Litterman for View 1
P1 = np.array([[0, 0, 0, -1, 0, 0, 0, 1, 0, 0]])
Q1 = np.array([0.06])

mubar1, Omegabar1, w_star1, Lambda1, U1 = blacklitterman(delta, w_M, Omega, tau, P1, Q1)

print("Posterior Risk Premia with View 1 (μ̄):\n", mubar1)
print("Optimal Portfolio Weights with View 1 (w*):\n", w_star1)

diff_mu1 = mubar1 - mueq
diff_w1 = w_star1 - w_M

print("Difference in Risk Premia (μ̄ - μ_eq) for View 1:\n", diff_mu1)
print("Difference in Portfolio Weights (w* - w_M) for View 1:\n", diff_w1)

Posterior Risk Premia with View 1 (μ̄):
 NoDur    0.130564
Durbl    0.175105
Manuf    0.145032
Enrgy    0.109614
HiTec    0.182602
Telcm    0.131380
Shops    0.154257
Hlth     0.133068
Utils    0.069498
Other    0.140581
dtype: float64
Optimal Portfolio Weights with View 1 (w*):
 [ 0.069       0.013       0.1196     -0.04540268  0.1914      0.0543
  0.0981      0.23470268  0.0446      0.2207    ]
Difference in Risk Premia (μ̄ - μ_eq) for View 1:
 NoDur   -0.000940
Durbl   -0.000017
Manuf   -0.004690
Enrgy   -0.023315
HiTec    0.001768
Telcm    0.003519
Shops    0.000062
Hlth     0.013231
Utils    0.000145
Other   -0.003449
dtype: float64
Difference in Portfolio Weights (w* - w_M) for View 1:
 [-1.00613962e-14  3.77649301e-15  1.11022302e-15 -1.41502679e-01
 -1.77635684e-15  1.66533454e-16  9.99200722e-16  1.41502679e-01
 -4.64905892e-16  2.47024623e-15]


We observe, as expected, an increase in the risk premium for the HiTec sector wheras the risk premium difference for Enrgy is negative. Reflecting the relative view, the porfolio weights that change most significantly are these two while the impact on other weights is relatively small.

# Part 3)

In [31]:
# Black-Litterman for Views 1 and 2
P2 = np.array([
    [0, 0, 0, -1, 0, 0, 0, 1, 0, 0],  # View 1
    [-0.1, 1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]  # View 2
])
Q2 = np.array([0.06, 0.04])

mubar2, Omegabar2, w_star2, Lambda2, U2 = blacklitterman(delta, w_M, Omega, tau, P2, Q2)

print("Posterior Risk Premia with Views 1 and 2 (μ̄):\n", mubar2)
print("Optimal Portfolio Weights with Views 1 and 2 (w*):\n", w_star2)

diff_mu2 = mubar2 - mueq
diff_w2 = w_star2 - w_M

print("Difference in Risk Premia (μ̄ - μ_eq) for Views 1 and 2:\n", diff_mu2)
print("Difference in Portfolio Weights (w* - w_M) for Views 1 and 2:\n", diff_w2)


Posterior Risk Premia with Views 1 and 2 (μ̄):
 NoDur    0.122237
Durbl    0.161693
Manuf    0.136201
Enrgy    0.106034
HiTec    0.173709
Telcm    0.126275
Shops    0.144980
Hlth     0.128869
Utils    0.067691
Other    0.133588
dtype: float64
Optimal Portfolio Weights with Views 1 and 2 (w*):
 [ 0.08198119 -0.11681194  0.13258119 -0.03481846  0.20438119  0.06728119
  0.11108119  0.25008085  0.05758119  0.23368119]
Difference in Risk Premia (μ̄ - μ_eq) for Views 1 and 2:
 NoDur   -0.009267
Durbl   -0.013430
Manuf   -0.013520
Enrgy   -0.026895
HiTec   -0.007125
Telcm   -0.001586
Shops   -0.009215
Hlth     0.009032
Utils   -0.001662
Other   -0.010441
dtype: float64
Difference in Portfolio Weights (w* - w_M) for Views 1 and 2:
 [ 0.01298119 -0.12981194  0.01298119 -0.13091846  0.01298119  0.01298119
  0.01298119  0.15688085  0.01298119  0.01298119]


We observe that the HiTech risk premium is adjusted downward compared to the case with just view 1. This suggests that the second view moderates the adjustment to the HiTec risk premium. In fact, we see a reduction in all risk premia aside from Hlth. This seems to indicate the market is pricing Durbl as if it will outperform the weighted average of other industries by even more than 4%. 